Code for peak shaving optimization

In [11]:
#import packages
import numpy as np
import pandas as pd
import cvxpy as cp
import gurobipy
from datetime import datetime

# Data pre-processing

In [17]:
#Add date parser to read in the date as an actual date object
date_parser = lambda x: datetime.strptime(x, '%d-%b-%Y')
demand = pd.read_csv('./data/ColumbiaDemand.csv', parse_dates=['Date'], date_parser=date_parser) 
demand.head()
#Restructure into one list of demands for each month

,Date,TotalDemand [kWh],Period01 [kWh],Period02 [kWh],Period03 [kWh],Period04 [kWh],Period05 [kWh],Period06 [kWh],Period07 [kWh],Period08 [kWh],...,Period87 [kWh],Period88 [kWh],Period89 [kWh],Period90 [kWh],Period91 [kWh],Period92 [kWh],Period93 [kWh],Period94 [kWh],Period95 [kWh],Period96 [kWh]
0,2018-01-01,72686.88,734.40,731.52,727.20,735.84,730.08,728.64,731.52,730.08,...,740.16,741.60,734.40,735.84,735.84,730.08,725.76,731.52,728.64,724.32
1,2018-01-02,73594.08,728.64,727.20,728.64,727.20,732.96,734.40,734.40,737.28,...,743.04,741.60,741.60,738.72,750.24,743.04,750.24,738.72,738.72,743.04
2,2018-01-03,73440.00,745.92,743.04,745.92,744.48,741.60,734.40,735.84,734.40,...,734.40,732.96,721.44,714.24,721.44,725.76,727.20,724.32,731.52,722.88
3,2018-01-04,73967.04,718.56,718.56,724.32,727.20,720.00,724.32,725.76,720.00,...,747.36,744.48,735.84,734.40,741.60,743.04,738.72,741.60,740.16,735.84
4,2018-01-05,74625.12,745.92,743.04,744.48,741.60,737.28,743.04,747.36,748.80,...,767.52,768.96,760.32,758.88,756.00,756.00,753.12,754.56,750.24,745.92


In [43]:
#Drop total demand columns and pivot
demand_unpivoted = demand[demand.columns.difference(['TotalDemand [kWh]'])].melt(id_vars=['Date'], var_name='period', value_name='demand')
#Make sure to sore by date and period
demand_unpivoted = demand_unpivoted.sort_values(by=['Date','period']).reset_index(drop=True)
demand_unpivoted['year'] = demand_unpivoted['Date'].apply(lambda x: x.year)
demand_unpivoted['month'] = demand_unpivoted['Date'].apply(lambda x: x.month)
demand_unpivoted.head() 

,Date,period,demand,year,month
0,2018-01-01,Period01 [kWh],734.40,2018,1
1,2018-01-01,Period02 [kWh],731.52,2018,1
2,2018-01-01,Period03 [kWh],727.20,2018,1
3,2018-01-01,Period04 [kWh],735.84,2018,1
4,2018-01-01,Period05 [kWh],730.08,2018,1


## Example of data selection for a single month (and year)

In [37]:
jan_2018 = demand_unpivoted[(demand_unpivoted['year']==2018) & (demand_unpivoted['month'] == 1)]
jan_2018.head()

,Date,period,demand,year,month
0,2018-01-01,Period01 [kWh],734.40,2018,1
1,2018-01-01,Period02 [kWh],731.52,2018,1
2,2018-01-01,Period03 [kWh],727.20,2018,1
3,2018-01-01,Period04 [kWh],735.84,2018,1
4,2018-01-01,Period05 [kWh],730.08,2018,1


In [57]:
jan_2018_demand = np.array(jan_2018['demand'])
jan_2018_demand

array([734.4 , 731.52, 727.2 , ..., 705.6 , 708.48, 711.36])

### Find the peak period for a month

In [60]:
# Add column for rolling sum of current row and preious for demand
jan_2018['rolling_demand'] = jan_2018[['demand']].rolling(2).sum()
jan_2018.head()

,Date,period,demand,year,month,rolling_demand
0,2018-01-01,Period01 [kWh],734.40,2018,1,NaN
1,2018-01-01,Period02 [kWh],731.52,2018,1,1465.92
2,2018-01-01,Period03 [kWh],727.20,2018,1,1458.72
3,2018-01-01,Period04 [kWh],735.84,2018,1,1463.04
4,2018-01-01,Period05 [kWh],730.08,2018,1,1465.92


In [88]:
#Print location where the demand is a maximum - get the period and see the charges
print('rolling max demand is ', jan_2018['rolling_demand'].max(), 'at index ',jan_2018['rolling_demand'].argmax())
print('The max periods are: ', jan_2018.loc[jan_2018['rolling_demand'].argmax()-1]['period'], ' and ', jan_2018.loc[jan_2018['rolling_demand'].argmax()]['period'])
print('Peak periods at hours: ', str(int(jan_2018.loc[jan_2018['rolling_demand'].argmax()-1]['period'].split('Period')[1].split(' ')[0])/4), ' and ', str(int(jan_2018.loc[jan_2018['rolling_demand'].argmax()]['period'].split('Period')[1].split(' ')[0])/4))
print('Look at the data at that period: ')
jan_2018[jan_2018['rolling_demand'].argmax()-2:jan_2018['rolling_demand'].argmax()+2]

rolling max demand is  1905.12 at index  720
The max periods are:  Period48 [kWh]  and  Period49 [kWh]
Peak periods at hours:  12.0  and  12.25
Look at the data at that period: 


,Date,period,demand,year,month,rolling_demand
718,2018-01-08,Period47 [kWh],947.52,2018,1,1889.28
719,2018-01-08,Period48 [kWh],953.28,2018,1,1900.80
720,2018-01-08,Period49 [kWh],951.84,2018,1,1905.12
721,2018-01-08,Period50 [kWh],953.28,2018,1,1905.12


# Optimization

In [4]:
#Define variables
days = 31 #28 or 30 depending on the month
periods = 96
# Battery Efficiency
eta = np.array([.95, .7]) #.95 or .7
# Power Rating
P = np.array([300, 500]) # 300 or 500
# Energy Rating
E = np.array([200, 50]) # 200 or 50
# Demand
# D = january_list, feb_list, ...
# Peak Demand Charge
# On peak month: June, July, August, September
#       8am-6pm, 8am-10pm, all-day
# Off peak month: other
#       8am-10pm, all-day
#Note: periods 32-72 are 8am-6pm, periods 32-88 are 8am-10pm
C = np.array([[9.15, 18.44, 16.66],
            [13.96, 4.21]])
# B is some selection of C and its summation
#Battery duration E*P -> if P=1, then E=4 or E=12, if P=2, then E=8 or E=24
bat_dur = np.array([4, 12])

# Decision Variables
#Storage discharge power
d = cp.Variable((days*periods), nonneg=True)
# Storage charge power
q = cp.Variable((days*periods), nonneg=True)
# Energy storged
e = cp.Variable((days*periods), nonneg=True)
# peak demand
p = cp.Variable(nonneg=True)


/var/folders/8w/yzz7q0p146x4qrt7dwp0mhy40000gn/T/ipykernel_69682/3273664124.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  C = np.array([[9.15, 18.44, 16.66],


In [ ]:
#Define Objective - BP + sum over t
obj = cp.Minimize(B*p + C*sum(D - d - q))